### Adafruit PCA9685 16-Channel Servo Tests

Checking out the servo comtrol board. You need to install the needed libraries using these commands in a terminal

```
sudo pip3 install adafruit-circuitpython-pca9685
sudo pip3 install adafruit-circuitpython-servokit
```



In [2]:
import time
from gamepad import Gamepad
from robotarm import Arm, Joint
from IPython.display import clear_output

## Robot arm configuration

In [3]:
# This is the robot arm configuration

config=[
    {'name': 'Rotation',   'channel': 0, 'max_angle': 170, 'min_angle': 10, 'home_angle': 90},
    {'name': 'Lower arm',  'channel': 1, 'max_angle': 140, 'min_angle': 45, 'home_angle': 90},
    {'name': 'Upper arm',  'channel': 2, 'max_angle': 120, 'min_angle': 20, 'home_angle': 90},
    {'name': 'Wrist tilt', 'channel': 3, 'max_angle': 175, 'min_angle': 50, 'home_angle': 90},
    {'name': 'Gripper',    'channel': 4, 'max_angle': 170, 'min_angle':  5, 'home_angle': 90}
]

In [4]:
# Instanciate the arm object
arm=Arm(config)

### Using GameContoller connected to Pi

This needs the code from https://github.com/piborg/Gamepad and the controller dongle is plugged into the middle USB port of the Raspberry Pi.

In [5]:
def updateJoint(joint, axis, speed):
    if(abs(gamepad.axis(axis)) > 0.1):
        arm.joints[joint].jog(speed*gamepad.axis(axis))

In [6]:
# Gamepad settings
running = True

gamepad = Gamepad()
print('Gamepad connected')

# Start the background updating
gamepad.startBackgroundUpdates()

# Handle joystick updates one at a time
try:
    start_time   = time.time()
    current_time = start_time
    
    while running and gamepad.isConnected():
        if gamepad.beenPressed('X'):         # 'X' exit control loop
            running = False
        if gamepad.beenPressed('B'):         # 'B' send arm to home position
            arm.home()
        if gamepad.beenPressed('A'):         # 'A' Save current location
            arm.save_location()
            print('Position saved')
        if gamepad.beenPressed('Y'):         # 'Y' Print current angles on screen
            clear_output(wait=True)
            print(arm)
            print(f"Saved positons - {arm.stored_locations()}")
        if gamepad.beenPressed('START'):     # 'START' replay saved postitions
            arm.run_locations()
        if gamepad.beenPressed('SELECT'):    # 'SELECT' clear saved postitions
            arm.clear_locations()
            print('Positons cleared')
            
        # Update all positions
        updateJoint(0, "LEFT-X", 1.5)        # Rotation
        updateJoint(1, "LEFT-Y", 1.5)        # Lower arm
        updateJoint(2, "RIGHT-X", 2.0)       # Upper arm
        updateJoint(3, "RIGHT-Y", 2.0)       # Wrist tilt
        updateJoint(4, "DPAD-Y", 2.0)        # Gripper

        time.sleep(0.01)
        
except KeyboardInterrupt:
    print ('Finished')
finally:
    gamepad.disconnect()
    print ('Disconnected')

Arm status
  0 at position 89.8
  1 at position 89.8
  2 at position 89.8
  3 at position 89.8
  4 at position 89.8
Saved positons - 1
Disconnected


In [ ]:
print(arm.locations)